In [ ]:
# default_exp funcs.predict

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from decode_fish.imports import *
from decode_fish.funcs.file_io import *
from decode_fish.funcs.emitter_io import *
from decode_fish.funcs.utils import *
from monai.inferers import sliding_window_inference

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
cfg = OmegaConf.load(f'../config/experiment/msp300_1.yaml')

In [ ]:
model, post_proc, psf, micro, img_3d, decode_dl = load_all(cfg, False)

/groups/turaga/home/speisera/anaconda3/envs/decode2_dev/lib/python3.7/site-packages/torch/cuda/__init__.py:81: UserWarning: 
    Found GPU2 NVS 510 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


Crop size larger than volume in at least one dimension. Crop size changed to (37, 48, 48)
1 volumes


In [ ]:
image_paths = sorted(glob.glob(str(Path(cfg.data_path.image_path).parent)+'/msp300*.tif'))
image_paths

['/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/datasets/CodFish/smFISH_data_Titlow/msp300_smFISH_1.tif',
 '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/datasets/CodFish/smFISH_data_Titlow/msp300_smFISH_2.tif',
 '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/datasets/CodFish/smFISH_data_Titlow/msp300_smFISH_3.tif',
 '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/datasets/CodFish/smFISH_data_Titlow/msp300_smFISH_4.tif']

In [ ]:
# export
def predict(model, post_proc, image_paths, window_size=[128,256,256], device='cuda'):
    
    pred_df = DF()
    with torch.no_grad():
        for p in tqdm(image_paths):
            print(p.split('/')[-1])
            img = load_tiff_image(p)
            z,y,x = img.shape[-3:]
            img = img.reshape(-1,z,y,x)
            for i in range(len(img)):
                output = sliding_window_inference(img[i][None,None], window_size, 1, model.to(device), overlap=0.2, sw_device=device, device='cpu', mode='gaussian')
                output = model.tensor_to_dict(output)
                p_si = sliding_window_inference(output['logits'], window_size, 1, post_proc, overlap=0.2, sw_device=device, device='cpu', mode='gaussian')
                i_df = post_proc.get_df(output, p_si)
                print('N. emitters: ', len(i_df))
                pred_df = append_emitter_df(pred_df, i_df)
                free_mem()
            
        return pred_df

In [ ]:
pred_df = predict(model, image_paths)

  0%|          | 0/4 [00:00<?, ?it/s]

msp300_smFISH_1.tif


  0%|          | 0/4 [00:02<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 3.00 GiB (GPU 0; 11.78 GiB total capacity; 3.28 GiB already allocated; 2.07 GiB free; 3.53 GiB reserved in total by PyTorch)

In [ ]:
with torch.no_grad():
    output = sliding_window_inference(img[None], [37,128,128], 1, model.cuda(), overlap=0.2, sw_device ='cuda', device='cpu', mode='gaussian')
    output = model.tensor_to_dict(output)
    p_si = sliding_window_inference(output['logits'], [37,128,128], 1, post_proc, overlap=0.2, sw_device ='cuda', device='cpu', mode='gaussian')
    pred_df = post_proc.get_df(output, p_si)

In [ ]:
with torch.no_grad():
    model.cpu()
    res = model.tensor_to_dict(model(img[None]))
    gt_df = post_proc.get_df(res)
    free_mem()

In [ ]:
gt_df[:10]

In [ ]:
pred_df[:10]

In [ ]:
print(len(gt_df))
print(len(pred_df))

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted 17_eval_routines.ipynb.
Converted 18_predict_funcs.ipynb.
Converted index.ipynb.
